In [ ]:
# Import neccessary libraries 
import requests 
import bs4 
import lxml 


# request html data, parse and inspect contents (for one movie)
result = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")
soup = bs4.BeautifulSoup(result.content, "lxml")


# get data from information box for movie 
box = soup.find(class_="infobox vevent")
info_rows = box.find_all("tr")
for row in info_rows:
    table_data = row.find_all("td")
    

# extract information for table_data key with multiple value

def get_row_info(table_data):
    if table_data.find("ul"):
        for li in table_data.find_all("ul"):
            text = [li.get_text().replace("\xa0", " ")]
            for string in text:
                new_text = string.split("\n")
            return new_text
    else: 
        return table_data.get_text().replace("\xa0", " ")

movie_info = {}
for index,row in enumerate(info_rows):
        if index == 0:
            movie_info["title"] = row.find("th").get_text(" ", strip=True)
        elif index == 1:
            pass 
        else: 
            table_key = row.find("th").get_text(" ", strip=True)
            table_value = get_row_info(row.find("td"))
            movie_info[table_key] = table_value
                        
movie_info            



## GET INFOBOX FOR ALL MOVIES  

result = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs4.BeautifulSoup(result.content, "lxml")
tables = soup.find_all(class_="wikitable sortable")



def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]): 
        tag.decompose()
        
        
# Function to get information(Director, cast, etc.) from every movie url on the list of disney movies 
def get_info_block(url):
    
    result = requests.get(url)
    soup = bs4.BeautifulSoup(result.content, "lxml")
    box = soup.find(class_="infobox vevent")
    info_rows = box.find_all("tr")
    clean_tags(soup)
    
    for row in info_rows:
        table_data = row.find_all("td")
    
    def get_row_info(table_data):
        
        if table_data.find("ul"):
            for li in table_data.find_all("ul"):
                text = [li.get_text().replace("\xa0", " ")]
                for string in text:
                    new_text = string.split("\n")
                return new_text
            
        elif table_data.find("br"):
            return [text for text in table_data.stripped_strings]    
            
        else: 
            return table_data.get_text().replace("\xa0", " ")
    
    
    movie_info = {}
    for index,row in enumerate(info_rows):
        if index == 0:
            movie_info["title"] = row.find("th").get_text(" ", strip=True)
        elif index == 1:
            pass 
        else: 
            table_key = row.find("th").get_text(" ", strip=True)
            table_value = get_row_info(row.find("td"))
            movie_info[table_key] = table_value
    
    return movie_info



result = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs4.BeautifulSoup(result.content, "lxml")
content = soup.prettify()


#identify movie urls on disney movies page

movies = soup.select(".wikitable.sortable i a")
print(len(movies))
movies[0]["href"]


# Loop through movie urls and apply function on every link 
movie_link_list = []
for index, movie in enumerate(movies):
    try:
        link = movie["href"]
        full_path = "https://en.wikipedia.org{}".format(link)
        movie_link_list.append(get_info_block(full_path))
        
    except Exception as e: 
        print(movie["title"])
        print(e)
        
# remove "minute" string and convert runtime to integer  
def conv_run_time(run_time):
    if type(run_time) == str:
        return int(run_time.split(" ")[0])
    elif type(run_time) == list:
        return int(run_time[0].split(" ")[0])
               
              
for movie in movie_link_list:
    runtime = movie.get("Running time")
    movie["Running time"] = conv_run_time(runtime)        



# #Save output as a JSON file
# import json
# json.dump(movie_link_list, open("disney_data.json", "w", encoding="utf-8"))

# # Convert JSON file into pandas DataFrame 
# import pandas as pd
# df = pd.read_json("disney_data.json", encoding="utf-8")
df

